In [1]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

# Replace these with your values
subscription_id = "1655f00d-a4c4-4ef2-84e1-fe175ad5cf99"
resource_group = "edforcerg"
workspace_name = "edforcews"

# Create an MLClient instance
credential = DefaultAzureCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)


In [2]:


# deploy the model to AKS
import datetime
from azure.ai.ml.entities import KubernetesOnlineEndpoint

online_endpoint_name= "k8s-endpoint"+ datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint= KubernetesOnlineEndpoint(
    name=online_endpoint_name,
    compute="k8s-compute",
    description="this is a sample k8s endpoint",
    auth_mode="key",
    tags={"key": "test_deplyoment"},
)






In [3]:
# then create the endpoint
ml_client.begin_create_or_update(endpoint).result()

KubernetesOnlineEndpoint({'provisioning_state': 'Succeeded', 'scoring_uri': 'http://85.210.121.2/api/v1/endpoint/k8s-endpoint11190938791381/score', 'openapi_uri': 'http://85.210.121.2/api/v1/endpoint/k8s-endpoint11190938791381/swagger.json', 'name': 'k8s-endpoint11190938791381', 'description': 'this is a sample k8s endpoint', 'tags': {'key': 'test_deplyoment'}, 'properties': {'createdBy': 'quickfix tech', 'createdAt': '2024-11-19T09:38:18.734879+0000', 'lastModifiedAt': '2024-11-19T09:38:18.734879+0000', 'azureml.onlineendpointid': '/subscriptions/1655f00d-a4c4-4ef2-84e1-fe175ad5cf99/resourcegroups/edforcerg/providers/microsoft.machinelearningservices/workspaces/edforcews/onlineendpoints/k8s-endpoint11190938791381', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/1655f00d-a4c4-4ef2-84e1-fe175ad5cf99/providers/Microsoft.MachineLearningServices/locations/uksouth/mfeOperationsStatus/oeidp:fd82a754-94a7-4b4c-83f0-f1cadb15311c:a4476f33-aead-42ec-a843-08f0746cf0c9?api-v

In [4]:


from azure.ai.ml.entities import KubernetesOnlineDeployment, CodeConfiguration

from azure.ai.ml.entities._deployment.resource_requirements_settings import ResourceRequirementsSettings
from azure.ai.ml.entities import Environment
from azure.ai.ml.entities import Model

# configure the deployment
model = Model(path=r"./model/diabetes_model.pkl")
env = Environment(
    conda_file=r"./environment/conda.yaml",
    image="mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest",
)





In [5]:
from azure.ai.ml.entities import ResourceSettings

from azure.ai.ml.entities import ResourceRequirementsSettings

blue_deployment = KubernetesOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code=r"./onlinescoring", scoring_script="score.py"
    ),
    instance_count=1,
    resources=ResourceRequirementsSettings(
        requests=ResourceSettings(
            cpu="100m",
            memory="0.5Gi",
        )
    ),
)


In [6]:
ml_client.begin_create_or_update(blue_deployment).result()

Check: endpoint k8s-endpoint11190938791381 exists
Uploading diabetes_model.pkl (< 1 MB): 100%|██████████| 21.4k/21.4k [00:00<00:00, 2.23MB/s]




.........